In [1]:
import json
import pandas as pd

In [4]:
# function to get shortened wiki articles

N_SENTENCES=3
def get_sentences(x):
    n = N_SENTENCES + 1
    sentences = x.split('. ')
    try:
        return '. '.join(sentences[:n])
    except:
        return '. '.join(sentences)
def count_sentenes(x):
    return len(x.split('. '))

In [2]:
# read pickles

wiki_en = pd.read_pickle('data/wiki_en_summaries.pkl')
wiki_fr = pd.read_pickle('data/wiki_fr_summaries.pkl')
bnf_summaries = pd.read_pickle('data/bnf_summaries.pkl')
named_subjects = pd.read_pickle('data/named_subjects.pkl')
age_gender = pd.read_json('age_gender_labeles.json',orient='records')

In [3]:
# get name-id df (exploded)

named_subjects = named_subjects[['id','title','names']].explode('names')
named_subjects = named_subjects.set_index('id')
named_subjects = named_subjects.rename(columns = {'names':'name'})

In [ ]:
# get id - 

id_img = named_subjects.drop('title', axis=1)
id_img = id_img.reset_index()
id_img = pd.DataFrame(id_img.groupby('name')['id'].apply(list))
id_img = id_img.rename(columns={'id':'id_list'})

In [ ]:
# organize dataframes, shorten texts, rename columns

wiki_en['wiki_en_text'] = wiki_en.summary.apply(get_sentences)
wiki_fr['wiki_fr_text'] = wiki_fr.summary_fr.apply(get_sentences)

wiki_en = wiki_en[['name','wiki_en_text','url','rank_en']]
wiki_fr = wiki_fr[['name','wiki_fr_text','url_fr','rank_fr']]
bnf_summaries = bnf_summaries[['raw_name','bnf_note','bnf_link']]

bnf_summaries = bnf_summaries.rename(columns={'raw_name':'name'})
wiki_en = wiki_en.rename(columns={'url':'wiki_en_link'})
wiki_fr = wiki_fr.rename(columns={'url':'wiki_fr_link'})

In [70]:
named_age_gender = named_subjects.merge(age_gender[['age','gender','id']], on='id')

In [71]:
data_final = pd.merge(bnf_summaries,wiki_en,how='outer')
data_final = pd.merge(data_final,wiki_fr,how='outer')
data_final = pd.merge(data_final, named_age_gender, how='outer')
data_final = data_final.merge(id_img, on='name',how='outer')

In [ ]:
data_final = data_final[['id','name','id_list', 'title','bnf_note', 'bnf_link', 'wiki_en_text', 'wiki_en_link',
       'rank_en', 'wiki_fr_text', 'url_fr', 'rank_fr', 'age','gender']]

In [72]:
data_final.to_pickle('data/merged_dataframe.pkl')